In [1]:
import numpy as np
import pandas as pd
import duckdb

## Using DuckDB to handle large dataset

In [9]:

# Read all CSVs (Feb 2021 to Sept 2021)
query = """
    SELECT * FROM read_csv_auto('../dataset/monthly_tweets/tweets_*.csv', 
        columns={'date': 'VARCHAR', 'text': 'VARCHAR', 'is_retweet': 'BOOLEAN'})
"""

# Run the query and get a DuckDB relation (can also `.to_df()` for pandas)
tweets_rel = duckdb.query(query)

# Optional: Create or connect to a DuckDB DB file
conn = duckdb.connect('bitcoin_tweets.duckdb')

# Create a persistent table (one-time operation)
conn.execute("""
    CREATE TABLE IF NOT EXISTS bitcoin_tweets AS
    SELECT * FROM read_csv_auto('../dataset/monthly_tweets/tweets_*.csv', 
        columns={'date': 'VARCHAR', 'text': 'VARCHAR', 'is_retweet': 'BOOLEAN'})
""")



In [ ]:
df = conn.execute("""
    SELECT * FROM bitcoin_tweets
    WHERE is_retweet = FALSE
    AND date BETWEEN '2021-02-01' AND '2021-07-31'
""").df()
# Saving the data into a dataframe but tweets are from feb to july only


In [14]:
print(df.shape)
print(df.isnull().sum())

(720268, 3)
date          0
text          0
is_retweet    0
dtype: int64


## Cleaning the data for VADER


In [15]:
import re

def preprocess_for_vader(text):
    # Remove URLs
    text = re.sub(r"http\S+|www.\S+", "", text)
    # Remove @mentions
    text = re.sub(r"@\w+", "", text)
    # Remove hashtags 
    text = re.sub(r"#", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [16]:
df['text'] = df['text'].apply(preprocess_for_vader)

print(df['text'].iloc[:10])

0    The Why Behind Microsoft’s $19 Billion Nuance ...
1    Make a change and an impart on yourself and ot...
2    What are the biggest shitcoins? crypto btc bnb...
3    crypto bitcoin cryptocurrency blockchain btc e...
4    [1D] Bitcoin market is weakly trending up curr...
5    🤝 Follow me on . Let's hunt for Bitcoins toget...
6    Time to get in CRO folks, massive bull run is ...
7    $ewt $btc $eth $dot $ada $link $snx $Inj $band...
8    BTC tops, maxis take profits, hedge alts, alts...
9    Live info FOREX Bitcoin BTC Gold backs off fur...
Name: text, dtype: object


In [17]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure reproducibility
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

# Apply language detection
df['lang'] = df['text'].astype(str).apply(detect_language)

# Show all unique languages detected
unique_languages = df['lang'].unique()

print("Unique languages detected in the dataset:")
print(unique_languages)


Unique languages detected in the dataset:
['en' 'de' 'it' 'af' 'pl' 'hr' 'fi' 'unknown' 'nl' 'ca' 'pt' 'da' 'cy'
 'so' 'es' 'fr' 'tl' 'ro' 'no' 'sv' 'id' 'tr' 'vi' 'et' 'sl' 'sw' 'ar'
 'fa' 'ja' 'hu' 'th' 'sk' 'sq' 'lv' 'lt' 'cs' 'ru']


In [23]:
def count_language_frequencies(df, lang_column='lang'):
    return df[lang_column].value_counts()

# Assuming df['lang'] has language codes
lang_counts = count_language_frequencies(df)

print("Language frequencies in the dataset:")
print(lang_counts)


Language frequencies in the dataset:
lang
en         706685
de           1777
ro           1565
fr           1396
pt           1259
cy           1061
nl            973
it            787
ca            751
tl            642
so            572
af            387
id            251
da            247
no            189
vi            186
unknown       183
fi            181
es            167
pl            161
tr            138
sv            135
sl            115
et            104
hr            102
hu             43
sw             40
ar             36
sq             35
cs             32
sk             22
ja             19
th             11
lv              8
lt              4
fa              3
ru              1
Name: count, dtype: int64


In [24]:
def keep_only_english_tweets(df, lang_column='lang'):
    return df[df[lang_column] == 'en'].copy()

# Filtered DataFrame with only English tweets
english_df = keep_only_english_tweets(df)

# check how many remain
print(f"Number of English tweets: {len(english_df)}")


Number of English tweets: 706685


In [31]:
english_df.to_csv('filtered_tweets_feb_to_july.csv', index=False)


## Assigning Labels to tweets Using VADER

In [32]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia=SentimentIntensityAnalyzer()

def get_sentiment_label(text):
    # Get the sentiment scores
    sentiment_scores = sia.polarity_scores(text)
    
    # Determine the sentiment label based on the compound score
    if sentiment_scores['compound'] >= 0.05:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aj281\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
df['sentiment'] = df['text'].apply(get_sentiment_label)
# Count the number of tweets in each sentiment category 
sentiment_counts = df['sentiment'].value_counts()  
print("Sentiment counts:")
print(sentiment_counts)

# Save the DataFrame with sentiment labels to a CSV file
df.to_csv('tweets_with_sentiment_feb_to_july.csv', index=False)

Sentiment counts:
sentiment
positive    368314
neutral     246296
negative    105658
Name: count, dtype: int64
